In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.support import expected_conditions as EC  # condiciones esperadas...

from selenium.webdriver import ActionChains as AC   # acciones encadenadas, rollo doble click

from selenium.webdriver.common.keys import Keys  # manejar teclas

import time

import pandas as pd
pd.set_option('display.max_columns', None)   
pd.set_option('display.max_rows', None)

from webdriver_manager.chrome import ChromeDriverManager

PATH=ChromeDriverManager().install()

import warnings
warnings.filterwarnings('ignore')

opciones=Options()

# quita la bandera de ser robot
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)

opciones.headless=False   # si True, no aparece la ventana (headless=no visible)

opciones.add_argument('--start-maximized')         # comienza maximizado

#opciones.add_extension('driver/adblock.crx')       # adblocker

opciones.add_argument('user-data-dir=selenium')    # mantiene las coockies

import Utils
from datetime import date

from decouple import config
user_spoty = config('user_spoty', default='')
pass_spoty = config('pass_spoty', default='')

from joblib import Parallel, delayed


In [2]:
#Cargamos los csv de los autores que hemos recopilados

autores1=pd.read_csv('../data/autores.csv')
autores2=pd.read_csv('../data/autores_2.csv')
autores3=pd.read_csv('../data/autores_3.csv')
lst=[autores1,autores2,autores3]
autores=pd.concat(lst)

In [22]:
csvs=['2017','2018','2019','2020','2021','2022']
autores=[]
lst_autores=[]
tmp=[]

#Pasamos la lista de los csvs y los cortamos por la coma para separar a los autores y los recorremos y guardamos
import re

for i in csvs:
    lista_globa_spoty=pd.read_csv(f'../data/lista{i}.csv')
    for autor in lista_globa_spoty.artist_names:
    
        autor.strip()
        palabra=autor.split(',')
    
    
        for i in range(len(palabra)):
            autores.append(palabra[i])
        
    lst_autores.append(autores)

In [23]:
#Unimos a todos los autores en un sola fila
lst_prueba=[]
lst_prueba.extend(lst_autores[0])
lst_prueba.extend(lst_autores[1])
lst_prueba.extend(lst_autores[2])
lst_prueba.extend(lst_autores[3])
lst_prueba.extend(lst_autores[4])
lst_prueba.extend(lst_autores[5])

len(lst_prueba)

4264248

In [24]:
lst_prueba=list(set(lst_prueba))

In [25]:
len(lst_prueba)

2915

In [19]:
#Guardamos los datos en un dataframe
uniooon=pd.DataFrame(lst_prueba, columns=['autor'])
uniooon.to_excel('../data/autores.xls',index=False,index_label=False)

In [41]:
#Esta función recopila los estilos de la página indicada por autor y lo guardamos

def extraer_autores(autor):
    
    driver=webdriver.Chrome(PATH)
    driver.get('https://everynoise.com/engenremap.html')
    
    time.sleep(2)
    iframe=driver.find_element(By.XPATH, '/html/body/iframe')

    driver.switch_to.frame(iframe)
    
    input_autor=driver.find_element(By.XPATH, '/html/body/form/input[1]')
    
    time.sleep(2)
    
    input_autor.send_keys(str(autor))  # escribe texto

    input_autor.send_keys(u'\ue007')  # tecla ENTER

    time.sleep(2)
    
    datos=driver.find_element(By.XPATH,'/html/body/div').text
    
    
    
    
    cadena_final=datos
    
    #cadena_final=pd.DataFrame(cadena_final)
    
    driver.quit()
    
    return cadena_final

In [53]:
#Ejecutamos la función 

driver=webdriver.Chrome(PATH)
driver.get('https://everynoise.com/engenremap.html')
    
time.sleep(2)
iframe=driver.find_element(By.XPATH, '/html/body/iframe')

driver.switch_to.frame(iframe)
ttt=[]
for i in lst_prueba[670:]:
    
    ttt.append(extraer_autores(i))

In [46]:
len(ttt)

670

In [48]:
#Cruzamos los datos del dataframe con los estilos y el de los autores


uniooon=pd.DataFrame(columns=['autor','estilo'])
for a,e in zip(lst_prueba[:671],ttt):
    uniooon=uniooon.append({'autor':a, 'estilo':e},ignore_index=True)

In [49]:
#Guardamos el dataframe con los dos datos
uniooon.to_csv('../data/autores_1.csv',index=False,index_label=False)

In [17]:
def shazam(canciones):
    try:
        driver=webdriver.Chrome(PATH)
        driver.get('https://www.shazam.com/es-es')

        time.sleep(4)

        driver.maximize_window()

        time.sleep(6)

        caja=driver.find_element(By.XPATH,'/html/body/div[4]/header/div/div[1]/div[2]/div[1]/div/div[1]/form/input')

        
        time.sleep(6)


        caja.send_keys(canciones)


        caja.send_keys(u'\ue007') 

        time.sleep(6)

        cancion=driver.find_element(By.XPATH,'/html/body/div[4]/header/div/div[1]/div[2]/div[1]/div/div[1]/form/div[2]/div[2]/div[3]/div[2]/a').click()

        time.sleep(6)

        can=driver.find_element(By.XPATH,'/html/body/div[4]/div/main/div/div[1]/div/article[2]/div[2]/div[2]/div[1]/div/div/h3').text
        
        time.sleep(4)
        driver.quit()
        division_canciones=canciones.split(',')
        
        dict_cancion={'cancion':division_canciones[0], 'artista': division_canciones[1], 'estilo': can}
        
        df=pd.DataFrame([dict_cancion])
    except:
        division_canciones=canciones.split(',')
        dict_cancion={'cancion':division_canciones[0], 'artista': division_canciones[1], 'estilo': 'error'}
        df=pd.DataFrame([dict_cancion])
        
        
    
    return df

In [8]:
canciones=pd.read_excel('../data/canciones.xls')


In [5]:
len(canciones)

6953

In [12]:
hol=shazam(canciones.aut[0])
hol

,cancion,artista,estilo
0,Starboy,the weeknd,R&B/Soul


In [4]:
canciones.aut[0]

'Starboy , the weeknd'

In [9]:
lst_canciones=canciones.aut

In [18]:
from tqdm.notebook import tqdm 
lst_df=Parallel(n_jobs=-1, verbose=True)(delayed(shazam)(cancion) for cancion in tqdm(lst_canciones[:1000]))

  0%|          | 0/1000 [00:00<?, ?it/s]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed: 26.1min
[Parallel(n_jobs=-1)]: Done 438 tasks      | elapsed: 58.5min
[Parallel(n_jobs=-1)]: Done 788 tasks      | elapsed: 104.3min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 131.5min finished


In [19]:
lst_df

[    cancion      artista    estilo
 0  Starboy    the weeknd  R&B/Soul,
    cancion            artista estilo
 0  Closer    the chainsmokers  Dance,
             cancion    artista estilo
 0  Let Me Love You    dj snake  Dance,
                                     cancion        artista estilo
 0  Rockabye (feat. Sean Paul & Anne-Marie)    clean bandit  Dance,
       cancion artista       estilo
 0  One Dance    drake  Hip-Hop/Rap,
       cancion artista       estilo
 0  Fake Love    drake  Hip-Hop/Rap,
       cancion      artista estilo
 0  24K Magic    bruno mars    Pop,
                                     cancion    artista estilo
 0  Don't Wanna Know (feat. Kendrick Lamar)    maroon 5    Pop,
           cancion        artista       estilo
 0  Black Beatles    rae sremmurd  Hip-Hop/Rap,
              cancion      artista    estilo
 0  I Feel It Coming    the weeknd  R&B/Soul,
                     cancion   artista estilo
 0  Chantaje (feat. Maluma)    shakira  error,
             

In [ ]:
lst_df1=Parallel(n_jobs=-1, verbose=True)(delayed(shazam)(cancion) for cancion in tqdm(lst_canciones[1000:2000]))

In [ ]:
lst_df2=Parallel(n_jobs=-1, verbose=True)(delayed(shazam)(cancion) for cancion in tqdm(lst_canciones[2000:3000]))

In [ ]:
lst_df3=Parallel(n_jobs=-1, verbose=True)(delayed(shazam)(cancion) for cancion in tqdm(lst_canciones[3000:4000]))

In [ ]:
lst_df4=Parallel(n_jobs=-1, verbose=True)(delayed(shazam)(cancion) for cancion in tqdm(lst_canciones[4000:5000]))

In [ ]:
lst_df5=Parallel(n_jobs=-1, verbose=True)(delayed(shazam)(cancion) for cancion in tqdm(lst_canciones[5000:6000]))

In [ ]:
lst_df6=Parallel(n_jobs=-1, verbose=True)(delayed(shazam)(cancion) for cancion in tqdm(lst_canciones[6000:6954]))